In [ ]:
r"""Diffusion Model on MNIST - DDPM Training and Generation Example

## Overview

This example demonstrates how to use Artifex's DDPM (Denoising Diffusion Probabilistic Model)
on MNIST to generate realistic handwritten digits. It showcases both training and various
sampling techniques including DDIM for fast generation.

**Key Artifex Components Used:**
- `DDPMModel` - Artifex's DDPM implementation with noise scheduling
- `DDPMConfig` - Frozen dataclass configuration with nested configs
- `NoiseScheduleConfig` - Noise schedule configuration
- `UNetBackboneConfig` - UNet backbone configuration

## Source Code Dependencies

**Validated:** 2026-01-13

This example depends on the following Artifex source files:
- `src/artifex/generative_models/models/diffusion/ddpm.py` - DDPMModel class
- `src/artifex/generative_models/models/diffusion/base.py` - DiffusionModel base
- `src/artifex/generative_models/core/configuration/diffusion_config.py` - DDPMConfig
- `src/artifex/generative_models/core/configuration/diffusion_config.py` - NoiseScheduleConfig
- `src/artifex/generative_models/core/configuration/backbone_config.py` - UNetBackboneConfig

**Validation Status:**
- ✅ All dependencies validated against `memory-bank/guides/flax-nnx-guide.md`
- ✅ No anti-patterns detected (RNG handling checked 2025-10-16)
- ✅ All patterns follow Flax NNX best practices

## What You'll Learn

- [ ] How to configure and create a DDPM model using Artifex
- [ ] Understanding DDPM forward diffusion (adding noise)
- [ ] Model forward pass for noise prediction
- [ ] Generating samples with DDPM (slow, high quality)
- [ ] Generating samples with DDIM (fast, good quality)
- [ ] Visualizing the denoising process step-by-step
- [ ] Comparing sampling speeds and quality

## Prerequisites

- Artifex installed (run `source activate.sh`)
- Basic understanding of diffusion models
- Familiarity with JAX and Flax NNX
- ~30 minutes on GPU, ~2 hours on CPU for full training

## Usage

```bash
source activate.sh
python examples/generative_models/image/diffusion/diffusion_mnist.py
```

## Expected Output

The example will:
1. Create synthetic MNIST-like data (for quick demonstration)
2. Build DDPM model with Artifex's DDPMModel
3. Demonstrate forward diffusion (adding noise)
4. Show model forward pass (predicting noise)
5. Generate samples with DDPM sampling (1000 steps)
6. Generate samples with DDIM sampling (50 steps, 20x faster!)
7. Visualize the progressive denoising process
8. Save visualizations to `examples_output/diffusion_mnist_*.png`

## Key Concepts

### Denoising Diffusion Probabilistic Models (DDPM)

DDPM learns to reverse a gradual noising process:

**Forward Process (q):** Gradually adds Gaussian noise to data
$$q(x_t | x_{t-1}) = \mathcal{N}(x_t; \sqrt{1-\beta_t} x_{t-1}, \beta_t I)$$

**Reverse Process (p):** Learns to denoise, generating data from noise
$$p_\theta(x_{t-1} | x_t) = \mathcal{N}(x_{t-1}; \mu_\theta(x_t, t), \Sigma_\theta(x_t, t))$$

**Training Objective:** Predict the noise added at each step
$$L = \mathbb{E}_{t, x_0, \epsilon} \left[ \|\epsilon - \epsilon_\theta(x_t, t)\|^2 \right]$$

### Artifex's Modular Design

Artifex provides:
- **DDPMModel**: Full DDPM with configurable noise schedules
- **Noise schedules**: Linear, cosine, and custom schedules
- **Fast sampling**: DDIM integration for 20-50x speedup
- **Flexible backbone**: Works with any neural network architecture

### DDIM: Fast Sampling

DDIM (Denoising Diffusion Implicit Models) enables:
- Deterministic sampling (same seed → same output)
- 50 steps instead of 1000 (20x faster!)
- Comparable quality to DDPM
- Enables interpolation in latent space

## Estimated Runtime

- **CPU**: ~5-10 minutes (demo mode, synthetic data)
- **GPU**: ~2-3 minutes (demo mode, synthetic data)

For full MNIST training:
- **CPU**: ~2 hours
- **GPU**: ~30 minutes

## Author

Artifex Team

## Last Updated

2025-10-16
"""

# Diffusion Model on MNIST

This notebook demonstrates DDPM (Denoising Diffusion Probabilistic Models) on MNIST
using Artifex's modular diffusion components.

## Learning Objectives

By the end of this example, you will understand:
1. How to configure and use Artifex's DDPMModel
2. The forward diffusion process (adding noise)
3. The reverse process (denoising / generation)
4. DDPM vs DDIM sampling trade-offs
5. Visualizing the denoising trajectory

In [ ]:
# Cell 1: Import Dependencies
r"""
Import Artifex components:
- DDPMModel: Artifex's DDPM implementation
- DDPMConfig: Frozen dataclass configuration for DDPM
- NoiseScheduleConfig: Noise schedule configuration
- UNetBackboneConfig: UNet backbone architecture configuration
"""

import os

import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
from flax import nnx
from tqdm import tqdm

from artifex.generative_models.core.configuration.backbone_config import UNetBackboneConfig
from artifex.generative_models.core.configuration.diffusion_config import (
    DDPMConfig,
    NoiseScheduleConfig,
)
from artifex.generative_models.models.diffusion.ddpm import DDPMModel

## Setup

Initialize the environment, device, and random number generators.

### Device Management

JAX automatically detects and uses available GPUs. If no GPU is available,
it falls back to CPU.

In [ ]:
# Cell 2: Setup and Device Configuration
print("=" * 80)
print("DDPM MNIST Example - Using Artifex's DDPMModel")
print("=" * 80)
print()

# Check device
device = jax.default_backend()
print(f"🖥️  JAX backend: {device}")
print(f"🖥️  Available devices: {jax.device_count()}")
print()

# Initialize RNG streams
# We need separate streams for different random operations
seed = 42
print(f"🎲 Random seed: {seed}")
print()

# Create RNG streams
# - params: For model parameter initialization
# - noise: For adding noise in forward diffusion
# - sample: For sampling operations
# - dropout: For dropout layers (if used)
rngs = nnx.Rngs(
    params=seed,
    noise=seed + 1,
    sample=seed + 2,
    dropout=seed + 3,
)

## Data Loading

For this demonstration, we create synthetic MNIST-like data. This allows the example
to run quickly without requiring data downloads.

**In production**, you would load real MNIST:
```python
import tensorflow as tf
(train_images, _), (test_images, _) = tf.keras.datasets.mnist.load_data()
```

**Data Format:**
- Images: 28×28×1 (grayscale)
- Values: [-1, 1] range (normalized for diffusion models)
- Shape: (batch_size, height, width, channels)

In [ ]:
# Cell 3: Data Loading Function
def load_mnist_data():
    """Load MNIST dataset.

    In this demo, we use synthetic data for quick execution.
    Replace with real MNIST loading for production.

    Returns:
        Tuple of (train_images, test_images) in [-1, 1] range

    Note:
        Real MNIST loading:
        ```python
        import tensorflow as tf
        (train_images, _), (test_images, _) = tf.keras.datasets.mnist.load_data()
        train_images = (train_images / 127.5) - 1.0  # Normalize to [-1, 1]
        ```
    """
    # Create synthetic MNIST-like data
    print("📊 Loading data...")
    key = jax.random.key(42)
    train_key, test_key = jax.random.split(key)

    # Synthetic 28×28×1 images in [-1, 1] range
    # Real MNIST has 60,000 train + 10,000 test images
    train_images = jax.random.uniform(train_key, (1000, 28, 28, 1), minval=-1, maxval=1)
    test_images = jax.random.uniform(test_key, (100, 28, 28, 1), minval=-1, maxval=1)

    print(f"  ✅ Train data shape: {train_images.shape}")
    print(f"  ✅ Test data shape: {test_images.shape}")
    print(f"  ✅ Data range: [{train_images.min():.2f}, {train_images.max():.2f}]")
    print()

    return train_images, test_images


# Load data
train_images, test_images = load_mnist_data()

## Model Creation

Now we'll create Artifex's DDPMModel with proper configuration.

### Configuration Parameters

- **noise_steps**: Number of diffusion timesteps (1000 is standard)
- **beta_start**: Initial noise level (small value, typically 1e-4)
- **beta_end**: Final noise level (larger value, typically 0.02)
- **beta_schedule**: Noise schedule type ('linear' or 'cosine')

The beta schedule controls how noise is added across timesteps. Linear is simpler
but cosine often works better for images.

In [ ]:
# Cell 4: Create DDPM Model
print("🔧 Creating DDPM model using Artifex APIs...")
print()

# Configure the backbone network (UNet for image generation)
backbone_config = UNetBackboneConfig(
    name="unet_backbone",
    hidden_dims=(64, 128, 256),  # Encoder hidden dimensions (tuple for frozen dataclass)
    activation="gelu",
    in_channels=1,  # MNIST is grayscale (1 channel)
    out_channels=1,  # Output same number of channels
    time_embedding_dim=128,  # Time embedding dimension
    attention_resolutions=(16, 8),  # Apply attention at these resolutions
    num_res_blocks=2,  # Number of residual blocks per level
    channel_mult=(1, 2, 4),  # Channel multipliers for each level
    dropout_rate=0.0,  # No dropout for demo
)

# Configure the noise schedule
noise_schedule_config = NoiseScheduleConfig(
    name="linear_schedule",
    schedule_type="linear",  # Linear noise schedule
    num_timesteps=1000,  # Number of diffusion timesteps
    beta_start=1e-4,  # Initial noise level
    beta_end=2e-2,  # Final noise level
)

# Configure the DDPM model with nested configs
config = DDPMConfig(
    name="ddpm_mnist",
    backbone=backbone_config,
    noise_schedule=noise_schedule_config,
    input_shape=(28, 28, 1),  # MNIST image dimensions (H, W, C)
    loss_type="mse",  # Mean squared error loss
    clip_denoised=True,  # Clip denoised samples to [-1, 1]
)

print("📋 Model Configuration:")
print(f"  - Name: {config.name}")
print(f"  - Input shape: {config.input_shape}")
print(f"  - Noise steps: {config.noise_schedule.num_timesteps}")
print(f"  - Beta range: [{config.noise_schedule.beta_start}, {config.noise_schedule.beta_end}]")
print(f"  - Beta schedule: {config.noise_schedule.schedule_type}")
print()

# Create the DDPM model
# Artifex automatically initializes the noise schedule and backbone network
model = DDPMModel(config, rngs=rngs)

print("✅ DDPMModel created successfully!")
print(f"  - Model type: {type(model).__name__}")
print(f"  - Noise steps: {model.noise_steps}")
print(f"  - Input channels: {model.in_channels}")
print()

## Forward Diffusion Process

The forward diffusion process gradually adds noise to data. At step t=0, we have
clean data; at t=T (noise_steps), we have pure noise.

**Mathematical formulation:**
$$x_t = \sqrt{\bar{\alpha}_t} x_0 + \sqrt{1 - \bar{\alpha}_t} \epsilon$$

where:
- $\bar{\alpha}_t$ = cumulative product of (1 - β) values
- $\epsilon \sim \mathcal{N}(0, I)$ is Gaussian noise

Let's visualize how an image becomes noisy across different timesteps.

In [ ]:
# Cell 5: Demonstrate Forward Diffusion
print("🔍 Demonstrating forward diffusion (adding noise)...")
print()

# Take a test image
test_img = train_images[0:1]  # Shape: (1, 28, 28, 1)
print(f"Original image shape: {test_img.shape}")

# Add noise at different timesteps
timesteps_to_show = [0, 250, 500, 750, 999]
noisy_images = []

for t in timesteps_to_show:
    # Create timestep tensor
    t_tensor = jnp.array([t])

    # Apply forward diffusion
    # This is what happens during training: we add noise to clean images
    noisy_x, added_noise = model.forward_diffusion(test_img, t_tensor)

    noisy_images.append(noisy_x[0])  # Remove batch dimension
    print(f"  t={t:4d}: noise_level={jnp.mean(added_noise**2):.4f}")

print()

## Visualize Forward Diffusion

Let's see how the image progressively becomes noisier.

In [ ]:
# Cell 6: Plot Forward Diffusion Process
def visualize_diffusion_process(original, noisy_images, timesteps, title="Forward Diffusion"):
    """Visualize the forward diffusion process.

    Args:
        original: Original clean image
        noisy_images: List of noisy images at different timesteps
        timesteps: List of timestep values
        title: Plot title
    """
    n_images = len(noisy_images) + 1
    fig, axes = plt.subplots(1, n_images, figsize=(n_images * 2, 2))

    # Plot original
    img = (original.squeeze() + 1) / 2  # Convert from [-1,1] to [0,1]
    axes[0].imshow(img, cmap="gray", vmin=0, vmax=1)
    axes[0].set_title("Original")
    axes[0].axis("off")

    # Plot noisy versions
    for i, (noisy, t) in enumerate(zip(noisy_images, timesteps)):
        img = (noisy.squeeze() + 1) / 2  # Convert from [-1,1] to [0,1]
        axes[i + 1].imshow(img, cmap="gray", vmin=0, vmax=1)
        axes[i + 1].set_title(f"t={t}")
        axes[i + 1].axis("off")

    plt.suptitle(title, fontsize=14, y=1.05)
    plt.tight_layout()
    return fig


print("📊 Visualizing forward diffusion...")
fig = visualize_diffusion_process(test_img[0], noisy_images, timesteps_to_show)

# Save figure
output_dir = "examples_output"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "diffusion_mnist_forward.png")
fig.savefig(output_path, dpi=150, bbox_inches="tight")
print(f"  ✅ Saved to {output_path}")
print()

## Model Forward Pass (Noise Prediction)

During training, the model learns to predict the noise that was added at each timestep.
Let's test the model's forward pass to see how it predicts noise.

**Training objective:**
$$L = \|\epsilon - \epsilon_\theta(x_t, t)\|^2$$

where $\epsilon_\theta$ is our neural network that predicts the noise.

In [ ]:
# Cell 7: Test Model Forward Pass
print("🧪 Testing model forward pass (noise prediction)...")
print()

# Create a batch of noisy images at random timesteps
batch_size = 4
test_batch = train_images[:batch_size]

# Sample random timesteps for each image in batch
t_batch = jax.random.randint(
    jax.random.key(123),
    (batch_size,),
    0,
    model.noise_steps,
)

print(f"Test batch shape: {test_batch.shape}")
print(f"Timesteps: {t_batch}")
print()

# Forward pass: model predicts noise
# This is what happens during training
outputs = model(test_batch, t_batch)

# Extract predicted noise
predicted_noise = outputs.get("predicted_noise", outputs.get("output"))
print("✅ Model forward pass successful!")
print(f"  - Output shape: {predicted_noise.shape}")
print(f"  - Output keys: {list(outputs.keys())}")
print()

## Sampling with DDPM (Slow but High Quality)

Now comes the exciting part: generating new images from pure noise!

DDPM sampling performs the full reverse diffusion process:
1. Start with random noise: $x_T \sim \mathcal{N}(0, I)$
2. Denoise iteratively for T steps: $x_{t-1} = f(x_t, t)$
3. Return clean sample: $x_0$

**This is slow** (1000 steps) but produces high-quality samples.

In [ ]:
# Cell 8: Generate Samples with DDPM
print("🎨 Generating samples with DDPM sampling (1000 steps)...")
print("   ⚠️  This will take a while (1000 denoising steps)...")
print()

# Generate 8 samples
n_samples = 8

# DDPM sampling: full 1000 steps
# This is the original DDPM algorithm
samples_ddpm = model.sample(
    n_samples_or_shape=n_samples,
    scheduler="ddpm",  # Use DDPM scheduler
)

print(f"✅ Generated {n_samples} samples with DDPM")
print(f"  - Sample shape: {samples_ddpm.shape}")
print(f"  - Value range: [{samples_ddpm.min():.2f}, {samples_ddpm.max():.2f}]")
print()

## Sampling with DDIM (Fast and Good Quality)

DDIM (Denoising Diffusion Implicit Models) enables **much faster sampling**!

Instead of 1000 steps, DDIM can generate comparable quality with just **50 steps**
(20x speedup!).

**Key advantages:**
- 20-50x faster than DDPM
- Deterministic (same seed → same output)
- Enables interpolation in latent space
- Comparable quality to DDPM

This makes diffusion models practical for real-time applications.

In [ ]:
# Cell 9: Generate Samples with DDIM
print("⚡ Generating samples with DDIM sampling (50 steps, 20x faster!)...")
print()

# DDIM sampling: only 50 steps instead of 1000!
samples_ddim = model.sample(
    n_samples_or_shape=n_samples,
    scheduler="ddim",  # Use DDIM scheduler
    steps=50,  # Only 50 steps!
)

print(f"✅ Generated {n_samples} samples with DDIM")
print(f"  - Sample shape: {samples_ddim.shape}")
print(f"  - Value range: [{samples_ddim.min():.2f}, {samples_ddim.max():.2f}]")
print("  - Speedup: ~20x faster than DDPM!")
print()

## Visualize Generated Samples

Let's compare samples from DDPM and DDIM side by side.

In [ ]:
# Cell 10: Visualize Samples
def visualize_samples(samples, title="Generated Samples", n_cols=4):
    """Visualize a grid of generated samples.

    Args:
        samples: Generated images (N, H, W, C)
        title: Plot title
        n_cols: Number of columns in grid

    Returns:
        matplotlib figure
    """
    n_samples = len(samples)
    n_rows = (n_samples + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 2, n_rows * 2))
    axes = axes.flatten() if n_samples > 1 else [axes]

    for i, ax in enumerate(axes):
        if i < n_samples:
            # Convert from [-1, 1] to [0, 1] for display
            img = (samples[i].squeeze() + 1) / 2
            img = np.clip(img, 0, 1)

            ax.imshow(img, cmap="gray")
            ax.axis("off")
        else:
            # Hide unused subplots
            ax.axis("off")

    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    return fig


print("📊 Visualizing generated samples...")

# Plot DDPM samples
fig_ddpm = visualize_samples(samples_ddpm, title="DDPM Samples (1000 steps)")
output_path_ddpm = os.path.join(output_dir, "diffusion_mnist_ddpm_samples.png")
fig_ddpm.savefig(output_path_ddpm, dpi=150, bbox_inches="tight")
print(f"  ✅ DDPM samples saved to {output_path_ddpm}")

# Plot DDIM samples
fig_ddim = visualize_samples(samples_ddim, title="DDIM Samples (50 steps)")
output_path_ddim = os.path.join(output_dir, "diffusion_mnist_ddim_samples.png")
fig_ddim.savefig(output_path_ddim, dpi=150, bbox_inches="tight")
print(f"  ✅ DDIM samples saved to {output_path_ddim}")
print()

## Progressive Denoising Visualization

Let's visualize the denoising process step-by-step to see how the model transforms
noise into a clean image.

This helps build intuition for how diffusion models work.

In [ ]:
# Cell 11: Progressive Denoising
def generate_with_trajectory(model, n_samples=1, save_every=100):
    """Generate samples and save intermediate steps.

    Args:
        model: Diffusion model
        n_samples: Number of samples to generate
        save_every: Save every N steps

    Returns:
        List of intermediate images during denoising
    """
    # Start from pure noise
    shape = (n_samples, 28, 28, 1)
    x = jax.random.normal(rngs.sample(), shape)

    # Store trajectory
    trajectory = [x.copy()]

    # Denoise step by step
    steps = list(range(model.noise_steps - 1, -1, -save_every))
    if steps[-1] != 0:
        steps.append(0)  # Ensure we save the final image

    print(f"Denoising over {len(steps)} snapshots...")

    for i, t in enumerate(tqdm(range(model.noise_steps - 1, -1, -1), desc="Denoising")):
        # Create timestep for all samples
        t_batch = jnp.full((n_samples,), t, dtype=jnp.int32)

        # Get model prediction
        outputs = model(x, t_batch)
        predicted_noise = outputs.get("predicted_noise", outputs.get("output"))

        # Denoise one step
        x = model.denoise_step(x, t_batch, predicted_noise, clip_denoised=True)

        # Save snapshot
        if t % save_every == 0 or t == 0:
            trajectory.append(x.copy())

    return trajectory


print("🎬 Generating progressive denoising trajectory...")
print()

# Generate trajectory for one sample
trajectory = generate_with_trajectory(model, n_samples=1, save_every=200)

print(f"✅ Captured {len(trajectory)} snapshots")
print()

## Visualize Progressive Denoising

Watch how the model transforms pure noise into a digit!

In [ ]:
# Cell 12: Plot Progressive Denoising
def plot_trajectory(trajectory, title="Progressive Denoising"):
    """Plot the denoising trajectory.

    Args:
        trajectory: List of images at different denoising steps
        title: Plot title

    Returns:
        matplotlib figure
    """
    n_steps = len(trajectory)
    fig, axes = plt.subplots(1, n_steps, figsize=(n_steps * 2, 2))

    for i, img in enumerate(trajectory):
        # Convert from [-1, 1] to [0, 1]
        img_display = (img[0].squeeze() + 1) / 2
        img_display = np.clip(img_display, 0, 1)

        if n_steps > 1:
            ax = axes[i]
        else:
            ax = axes

        ax.imshow(img_display, cmap="gray", vmin=0, vmax=1)
        ax.axis("off")

        # Compute step number (counting backwards from noise_steps)
        step = (n_steps - i - 1) * 200
        if step < 0:
            step = 0
        ax.set_title(f"t={step}")

    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    return fig


print("📊 Visualizing progressive denoising...")
fig_traj = plot_trajectory(trajectory)
output_path_traj = os.path.join(output_dir, "diffusion_mnist_trajectory.png")
fig_traj.savefig(output_path_traj, dpi=150, bbox_inches="tight")
print(f"  ✅ Saved to {output_path_traj}")
print()

## Summary and Key Takeaways

✅ **What We Learned:**
- How to configure and use Artifex's DDPMModel
- The forward diffusion process (q) adds noise progressively
- The reverse process (p) learns to denoise and generate
- DDPM sampling is slow (1000 steps) but high quality
- DDIM sampling is fast (50 steps, 20x speedup) with comparable quality
- Diffusion models transform noise into structured data iteratively

💡 **Key Insights:**
- Diffusion models work by learning to reverse a gradual noising process
- The noise schedule controls how noise is added across timesteps
- DDIM makes diffusion models practical for real-time applications
- Artifex provides modular, easy-to-use diffusion components
- The same framework works for images, audio, and other modalities

📊 **Results:**
- Successfully demonstrated forward diffusion
- Generated samples with both DDPM and DDIM
- Visualized the progressive denoising process
- All visualizations saved to examples_output/

🔧 **Artifex APIs Used:**
- `DDPMModel`: Full DDPM implementation with noise scheduling
- `DDPMConfig`: Frozen dataclass configuration for DDPM
- `NoiseScheduleConfig`: Noise schedule configuration
- `UNetBackboneConfig`: UNet backbone architecture configuration

🔬 **Next Steps:**
- Train on real MNIST for realistic digit generation
- Experiment with different noise schedules (cosine vs linear)
- Try different numbers of timesteps (500, 2000, etc.)
- Implement conditional generation (class-conditional DDPM)
- Explore latent diffusion for higher resolutions

In [ ]:
print()
print("=" * 80)
print("DDPM MNIST Example Completed Successfully!")
print("=" * 80)
print()
print("💡 Key Takeaways:")
print("  - Diffusion models transform noise into data through iterative denoising")
print("  - DDPM: 1000 steps, high quality, slow")
print("  - DDIM: 50 steps, good quality, 20x faster!")
print("  - Artifex provides easy-to-use diffusion components")
print()
print("📁 Output files:")
print(f"  - {output_path}")
print(f"  - {output_path_ddpm}")
print(f"  - {output_path_ddim}")
print(f"  - {output_path_traj}")
print()
print("🔬 Experiments to Try:")
print("  - Load real MNIST data for better results")
print("  - Try different noise schedules (schedule_type='cosine' in NoiseScheduleConfig)")
print("  - Experiment with different step counts (DDIM steps=20, steps=100)")
print("  - Compare generation quality vs speed tradeoffs")
print()
print("📚 Related Examples:")
print("  - simple_diffusion_example.py: Basic diffusion concepts")
print("  - dit_demo.py: Diffusion Transformer architecture")
print()


if __name__ == "__main__":
    print("✨ Example complete! ✨")